In [23]:
import numpy as np
import pandas as pd
import torch
import torch as t, torch.nn as nn,  torch.nn.functional as f
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
df1 = pd.read_csv("/content/MBA_ADMISSIONS.csv")

df1.sample(6)

,pre_score,Age_in_years,Percentage_in_10_Class,Percentage_in_12_Class,Percentage_in_Under_Graduate,percentage_MBA,post_score,Gender,STATE,Previous_Degree,Marital_status,Place_you_belong_to,perceived#Job#Skill,Specialization
420,60.000000,20,93.00,92.0,77.00,69.00,85.000000,Female,South Zone,Science,Single,Rural,prefered skills,LOS
22,85.000000,26,86.40,80.4,70.22,70.48,76.666667,Male,Central Zone,Engineering,Single,Semi Urban,desired skills,LOS
270,40.000000,26,88.16,84.3,74.20,66.00,66.666667,Male,South Zone,Engineering,Single,Urban,prefered skills,Marketing
91,76.666667,25,83.40,79.0,66.80,66.10,80.000000,Male,North Zone,Engineering,Single,Urban,desired skills,Marketing
400,46.000000,23,85.80,84.6,89.00,69.40,100.000000,Male,South Zone,Engineering,Single,Urban,desired skills,HR
50,43.000000,22,78.00,80.0,87.00,67.27,100.000000,Male,East Zone,Engineering,Single,Urban,prefered skills,Finance


# Pre-Processing

In [3]:
df1.isnull().sum()

pre_score                       0
Age_in_years                    0
Percentage_in_10_Class          0
Percentage_in_12_Class          0
Percentage_in_Under_Graduate    0
percentage_MBA                  0
post_score                      0
Gender                          0
STATE                           0
Previous_Degree                 0
Marital_status                  0
Place_you_belong_to             0
perceived#Job#Skill             0
Specialization                  0
dtype: int64

In [4]:
for i in df1.select_dtypes(include=['object']):
  print(f'{i}: {len(df1[i].unique())} labels')

Gender: 2 labels
STATE: 6 labels
Previous_Degree: 5 labels
Marital_status: 2 labels
Place_you_belong_to: 3 labels
perceived#Job#Skill: 3 labels
Specialization: 4 labels


In [5]:
for i in df1.select_dtypes(include=['object']):
  print(f"{i}:\n{df1[i].value_counts()}\n-------------------------------------")

Gender:
Gender
Male      268
Female    205
Name: count, dtype: int64
-------------------------------------
STATE:
STATE
South Zone      258
North Zone       86
Central Zone     56
East Zone        52
West Zone        16
North East        5
Name: count, dtype: int64
-------------------------------------
Previous_Degree:
Previous_Degree
Engineering    235
Commerce       148
Management      44
Science         43
Arts             3
Name: count, dtype: int64
-------------------------------------
Marital_status:
Marital_status
Single     462
Married     11
Name: count, dtype: int64
-------------------------------------
Place_you_belong_to:
Place_you_belong_to
Urban         313
Semi Urban    116
Rural          44
Name: count, dtype: int64
-------------------------------------
perceived#Job#Skill:
perceived#Job#Skill
prefered skills    295
desired skills     167
required skills     11
Name: count, dtype: int64
-------------------------------------
Specialization:
Specialization
Marketing    17

In [6]:
x = df1.drop('Specialization', axis=1)
y = df1[['Specialization']]

In [7]:
#ordinal Encoder-For all object columns
oe = OrdinalEncoder()

def ConvertCategorical(f1):
  for column in f1.select_dtypes(include=["object"]):
    f1[column] = oe.fit_transform(f1[[column]])

ConvertCategorical(x)

#label Encoder - For the target column
y = LabelEncoder().fit_transform(y)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
x.head(10)

,pre_score,Age_in_years,Percentage_in_10_Class,Percentage_in_12_Class,Percentage_in_Under_Graduate,percentage_MBA,post_score,Gender,STATE,Previous_Degree,Marital_status,Place_you_belong_to,perceived#Job#Skill
0,75.000000,22,71.0,74.8,72.00,61.00,83.333333,1.0,0.0,2.0,1.0,2.0,1.0
1,71.666667,25,77.6,82.6,76.90,66.85,76.666667,1.0,0.0,2.0,1.0,1.0,1.0
2,76.666667,26,93.2,83.8,77.00,74.97,75.000000,0.0,0.0,2.0,1.0,2.0,0.0
3,66.666667,22,91.2,80.0,67.00,68.30,60.000000,1.0,0.0,1.0,1.0,1.0,1.0
4,71.666667,24,79.8,61.6,60.33,69.28,76.666667,0.0,0.0,2.0,1.0,2.0,1.0
5,83.333333,23,67.0,62.0,67.00,63.00,80.000000,0.0,0.0,1.0,1.0,2.0,0.0
6,73.333333,22,82.0,70.0,76.50,67.00,81.666667,1.0,0.0,2.0,1.0,1.0,1.0
7,85.000000,26,86.4,80.4,70.22,70.48,76.666667,1.0,0.0,2.0,1.0,1.0,0.0
8,71.666667,22,95.0,78.6,75.43,68.54,88.333333,1.0,0.0,2.0,1.0,2.0,1.0
9,75.000000,22,71.0,74.8,72.00,61.00,83.333333,1.0,0.0,2.0,1.0,2.0,1.0


# Model Building

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.8, random_state=1)

print("X_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (378, 13)
y_train shape: (378,)
X_test shape: (95, 13)
y_test shape: (95,)


In [10]:
x_train = t.FloatTensor(x_train.to_numpy())
y_train = t.FloatTensor(y_train)
x_test = t.FloatTensor(x_test.to_numpy())
y_test = t.LongTensor(y_test)

In [11]:
class regres(nn.Module):
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(13,8)
    # self.l2 = nn.Linear(21,8)
    self.o = nn.Linear(8,4)
    # self.a1 = nn.Softmax(dim=1)

  def forward(self,x):
    x = f.relu(self.l1(x))
    # x = f.relu(self.l2(x))
    x = f.relu(self.o(x))
    # x = self.a1(x)
    return x

classify = regres()
classify

regres(
  (l1): Linear(in_features=13, out_features=8, bias=True)
  (o): Linear(in_features=8, out_features=4, bias=True)
)

In [21]:
loss_fxn = nn.CrossEntropyLoss()

# optim1 = t.optim.SGD(classify.parameters(),lr=0.01, momentum=0.9)

def accuracy(y1,y2):
  t1 = t.eq(y1,y2).sum().item()
  return (t1/len(y1))*100

#Part - 1

Gradient Descent

In [13]:
class GradientDescent:
    def __init__(self, parameters, lr=0.01):
        self.parameters = list(parameters)
        self.lr = lr

    def zero_grad(self):
        for param in self.parameters:
            if param.grad is not None:
                param.grad.detach_()
                param.grad.zero_()

    def step(self):
        for param in self.parameters:
            if param.grad is not None:
                param.data -= self.lr * param.grad

gd = GradientDescent(classify.parameters(),lr=0.001)

In [ ]:
epochs = 10
batch_size = 30

for epoch in range(epochs):
  running_loss = 0.0
  classify.train()
  for i in range(0, len(x_train), batch_size):
    inputs = x_train[i:i+batch_size]
    labels = y_train[i:i+batch_size]
    optimSGD_mom.zero_grad()
    outputs = classify(inputs)
    print(outputs.shape,labels.shape)
    loss = loss_fxn(outputs, labels)
    loss.backward()
    optimSGD_mom.step()
    running_loss += loss.item()
  print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss/len(x_train)}")
  classify.eval()
  with torch.no_grad():
    outputs = classify(x_test)
    _, predicted = torch.max(outputs.data, 1)
    acc = accuracy_score(predicted, y_test)
    print(f"Accuracy: {100*acc:.2f}%")


Custom SGD with mom

In [30]:
class CustomSGD:
    def __init__(self, parameters, lr=0.01, momentum=0):
        self.parameters = list(parameters)
        self.lr = lr
        self.momentum = momentum
        self.velocity = [t.zeros_like(param.data) for param in self.parameters]

    def zero_grad(self):
        for param in self.parameters:
            if param.grad is not None:
                param.grad.data.zero_()

    def step(self):
        for i, param in enumerate(self.parameters):
            gradient = param.grad
            if gradient is not None:
                if self.momentum != 0:
                    self.velocity[i] = self.momentum * self.velocity[i] + (1 - self.momentum) * gradient
                    update = self.lr * self.velocity[i]
                else:
                    update = self.lr * gradient
                param.data -= update

optimSGD_mom = CustomSGD(classify.parameters(), lr=0.01, momentum=0.9)

In [ ]:
t.manual_seed(42)

losses=[]

for epoch in range(1):
    #
    classify.train()
    y_value = classify(x_train)
    #
    loss = loss_fxn(y_value,y_train)
    # print(loss)
    #
    optimSGD_mom.zero_grad()
    loss.backward()
    optimSGD_mom.step()
    #
    print(y_train,y_value,np.argmax(y_value.long()))
    acc = accuracy(y_train,np.argmax(y_value.long()))

    losses.append(loss)

    if epoch % 10 == 9:
        print(f"Epoch: {epoch+1} | Loss: {loss}")

SGD without momentum

In [43]:
class SGD:
    def __init__(self, parameters, lr=0.01):
        self.parameters = list(parameters)
        self.lr = lr

    def zero_grad(self):
        for param in self.parameters:
            if param.grad is not None:
                param.grad.detach_()
                param.grad.zero_()

    def step(self):
        for param in self.parameters:
            if param.grad is not None:
                param.data -= self.lr * param.grad


optimSGD = SGD(classify.parameters(), lr=0.01)

In [ ]:
t.manual_seed(42)

losses=[]

for epoch in range(100):
    #
    classify.train()
    y_value = classify(x_train)
    #
    loss = loss_fxn(y_value,y_train)
    # print(loss)
    #
    optimSGD.zero_grad()
    loss.backward()
    optimSGD.step()
    #
    acc = accuracy_score(y_train,y_pred.long())

    losses.append(loss)

    if epoch % 10 == 9:
        print(f"Epoch: {epoch+1} | Loss: {loss}")

RMS Prop

In [ ]:
class SGD_RMSprop:
    def __init__(self, parameters, lr=0.01, alpha=0.9, eps=1e-8):
        self.parameters = list(parameters)
        self.lr = lr
        self.alpha = alpha
        self.eps = eps
        self.squared_gradients = [t.zeros_like(param.data) for param in self.parameters]

    def zero_grad(self):
        for param in self.parameters:
            if param.grad is not None:
                param.grad.detach_()
                param.grad.zero_()

    def step(self):
        for i, param in enumerate(self.parameters):
            if param.grad is not None:
                gradient = param.grad
                self.squared_gradients[i] = self.alpha * self.squared_gradients[i] + (1 - self.alpha) * gradient**2
                update = self.lr * gradient / (t.sqrt(self.squared_gradients[i]) + self.eps)
                param.data -= update

rms_prop = SGD_RMSprop(classify.parameters())

In [ ]:
t.manual_seed(42)

losses=[]

for epoch in range(100):
    #
    classify.train()
    y_value = classify(x_train)
    #
    loss = loss_fxn(y_value,y_train)
    # print(loss)
    #
    rms_prop.zero_grad()
    loss.backward()
    rms_prop.step()
    #
    acc = accuracy_score(y_train,y_pred.long())

    losses.append(loss)

    if epoch % 10 == 9:
        print(f"Epoch: {epoch+1} | Loss: {loss}")

# Part 2

In [ ]:
##Gradient Descent contour plot

def loss(w1, w2):
  return (w1 - 2)**2 + (w2 - 1)**2

w1_range = np.linspace(0, 4, 100)
w2_range = np.linspace(0, 4, 100)
W1, W2 = np.meshgrid(w1_range, w2_range)

Z = loss(W1, W2)

plt.contour(W1, W2, Z, levels=20)

w1_history = [0, 1, 2, 3]
w2_history = [0, 2, 1, 4]
plt.plot(w1_history, w2_history, '-o', color='red')

plt.xlabel('w1')
plt.ylabel('w2')
plt.show()


In [ ]:

def loss(w1, w2):
  return (w1 - 2)**2 + (w2 - 1)**2

w1_range = np.linspace(0, 4, 100)
w2_range = np.linspace(0, 10, 50)
W1, W2 = np.meshgrid(w1_range, w2_range)

Z = loss(W1, W2)

plt.contour(W1, W2, Z, levels=20)

w1_history = [0, 1, 2, 3]
w2_history = [0, 2, 1, 4]
plt.plot(w1_history, w2_history, '-o', color='red')

plt.xlabel('w1')
plt.ylabel('w2')
plt.show()


In [ ]:

def loss(w1, w2):
  return (w1 - 2)**2 + (w2 - 1)**2

w1_range = np.linspace(0, 10, 100)
w2_range = np.linspace(0, 7, 50)
W1, W2 = np.meshgrid(w1_range, w2_range)

Z = loss(W1, W2)

plt.contour(W1, W2, Z, levels=20)

w1_history = [0, 1, 2, 3]
w2_history = [0, 2, 1, 4]
plt.plot(w1_history, w2_history, '-o', color='red')

plt.xlabel('w1')
plt.ylabel('w2')
plt.show()
